# Does Specializing More than Once Improve Learning

In [1]:
from res_specialization import *
from matplotlib import pyplot as plt
import sys
import random
import pickle
plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
TRIALS      = 1000
NUM_TO_SPEC = 3
TOL = 5

diff_eq_params = {"x0": [-20, 10, -.5], 
                  "begin": 0, 
                  "end": 60, 
                  "timesteps":60000}

res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

results = dict()


def save_results():
    pickle.dump(results, open("spec_tw.pkl","wb"))
# end

### Model trajectory

In [3]:
t, train_t, u = lorenz_equ(**diff_eq_params )
train_t = t[30000:55000]
test_t  = t[55000:]

### Re-run grant figure trials to get prediction length


In [4]:
random_err       = []
random_pred      = []
specialized_err  = []
specialized_pred = []


spec_sizes       = []
spec_edges       = []

for i in range(TRIALS):
    
    ######################
    # Random graph
    ######################
    
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Make predicitions
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store Error
    random_err.append(L2error(u(train_t), train_pre))
    random_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    
    #######################
    # Specialize graph
    ######################
    
    A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
    spec_sizes.append(A.shape[0])
    spec_edges.append(np.sum(A != 0))
    
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    specialized_err.append(L2error(u(train_t), train_pre))
    specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [5]:
results["random_err"]       = random_err
results["random_pred"]      = random_pred
results["specialized_err"]  = specialized_err
results["specialized_pred"] = specialized_pred
results["spec_sizes"]       = spec_sizes
results["spec_edges"]       = spec_edges
save_results()
print("Mean Prediction Length: {}".format(np.mean(random_pred)))
print("Mean Prediction Length: {}".format(np.mean(specialized_pred)))

Mean Prediction Length: 183.784
Mean Prediction Length: 2420.792


### Control: Same sizes and same edge densities as previous experiment

In [6]:
match_edge_err = []
match_edge_pred = []

for s,e in zip(spec_sizes,spec_edges):
    # Test random graph with same number of edges as specialized graphs
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    match_edge_err.append(L2error(u(train_t), train_pre))
    match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [7]:
results["match_edge_err"] = match_edge_err
results["match_edge_pred"] = match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(match_edge_pred)))
save_results()


Mean Prediction Length: 1426.149


### Specialization without finding best nodes

In [ ]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

random_specialized_err  = []
random_specialized_pred = []
rspec_sizes         = []
rspec_edges         = []

for i in range(TRIALS):
    # Random graph
    rc = ResComp(3,3,**res_params)
    
    # Turn reservoir to integer adj matrix
    A = rc.res
    for j in range(A.shape[0]): A[j,j] = 0
    A = (A != 0)*1
    
    # Specialize the reservoir
    A = specializeGraph(A, random.sample(list(range(30)),27))
    
    # Store data
    rspec_sizes.append(A.shape[0])
    rspec_edges.append(np.sum(A != 0))
    
    # Make new reservoir
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Get predicitons
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store data
    random_specialized_err.append(L2error(u(train_t), train_pre))
    random_specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [9]:
results["random_specialized_err"]  = random_specialized_err
results["random_specialized_pred"] = random_specialized_pred
results["rspec_sizes"]             = rspec_sizes
results["rspec_edges"]             = rspec_edges

print("Mean Prediction Length: {}".format(np.mean(random_specialized_pred)))
save_results()


Mean Prediction Length: 1999.188


### Control: Same sizes and same edge densities as previous experiment


In [14]:
rand_match_edge_err  = []
rand_match_edge_pred = []

for s,e in zip(rspec_sizes, rspec_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    rand_match_edge_err.append(L2error(u(train_t), train_pre))
    rand_match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    

In [15]:
results["rand_match_edge_err"]  = rand_match_edge_err
results["rand_match_edge_pred"] = rand_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(rand_match_edge_pred)))
save_results()


Mean Prediction Length: 1137.772


### Targeted Specialize Twice

In [17]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

spec_tw_pre = []
spec_tw_err = []
tw_edges = []
tw_sizes = []
HOW_MANY_SPEC = 2

for i in range(TRIALS):
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 2 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        rc.fit(train_t,u)
    
    # Predict the system states
    tw_sizes.append(A.shape[0])
    tw_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_tw_err.append(L2error(u(train_t), train_pre))
    spec_tw_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))

In [18]:
results["spec_tw_pre"] = spec_tw_pre
results["spec_tw_err"] = spec_tw_err
results["tw_edges"] = tw_edges
results["tw_sizes"] = tw_sizes
save_results()


In [20]:
print("Mean Prediction Length: {}".format(np.mean(spec_tw_pre)))


Mean Prediction Length: 2551.899


### Control: Same sizes and same edge densities as previous experiment

In [21]:
tw_match_edge_err  = []
tw_match_edge_pred = []

for s,e in zip(tw_sizes, tw_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    tw_match_edge_err.append(L2error(u(train_t), train_pre))
    tw_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [24]:
results["tw_match_edge_err"] = tw_match_edge_err
results["tw_match_edge_pred"] = tw_match_edge_pred
save_results()
print("Mean Prediction Length: {}".format(np.mean(tw_match_edge_pred)))


Mean Prediction Length: 1495.112


### Targeted Specialize Three Times

In [25]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

spec_thr_pre = []
spec_thr_err = []
thr_edges = []
thr_sizes = []
HOW_MANY_SPEC = 3

for i in range(TRIALS):
    # Make RC
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 3 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        rc.fit(train_t,u)
    
    # Make predicitons
    thr_sizes.append(A.shape[0])
    thr_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_thr_err.append(L2error(u(train_t), train_pre))
    spec_thr_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))


In [26]:
results["spec_thr_pre"] = spec_thr_pre
results["spec_thr_err"] = spec_thr_err
results["thr_edges"] = thr_edges
results["thr_sizes"] = thr_sizes
print("Mean Prediction Length: {}".format(np.mean(spec_thr_pre)))

save_results()


Mean Prediction Length: 2648.014


### Control: Same sizes and same edge densities as previous experiment

In [27]:
thr_match_edge_err  = []
thr_match_edge_pred = []

for s,e in zip(thr_sizes, thr_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Predict systems
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    thr_match_edge_err.append(L2error(u(train_t), train_pre))
    thr_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [29]:
results["thr_match_edge_err"] = thr_match_edge_err
results["thr_match_edge_pred"] = thr_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(thr_match_edge_pred)))

save_results()


Mean Prediction Length: 1537.555


## Organize results into dataframes


In [31]:
import pandas as pd

In [46]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
df = pd.DataFrame(results)
df.describe()

,random_err,random_pred,specialized_err,specialized_pred,spec_sizes,spec_edges,match_edge_err,match_edge_pred,random_specialized_err,random_specialized_pred,...,tw_edges,tw_sizes,tw_match_edge_err,tw_match_edge_pred,spec_thr_pre,spec_thr_err,thr_edges,thr_sizes,thr_match_edge_err,thr_match_edge_pred
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,15750.659529,183.784000,21.859027,2420.792000,117.102000,236.973000,276.229631,1426.149000,38.964745,1999.188000,...,286.363000,145.807000,146.741324,1495.11200,2648.014000,14.744624,362.180000,190.788000,72.924765,1537.555000
std,20722.656793,294.953391,77.936329,868.727774,74.618533,120.622984,1336.242214,879.593553,201.409312,995.228954,...,154.501352,89.702666,739.245195,902.49904,825.589575,10.445820,248.949214,144.561292,310.641665,829.118688
min,14.795582,0.000000,5.390908,98.000000,33.000000,86.000000,4.479763,0.000000,5.620447,42.000000,...,81.000000,39.000000,4.291349,0.00000,281.000000,7.074156,105.000000,52.000000,5.635701,27.000000
25%,84.051292,45.000000,13.604474,2041.000000,73.000000,166.000000,14.141500,685.000000,15.155787,1324.000000,...,192.000000,89.000000,12.788221,702.50000,2108.750000,12.388319,226.000000,112.000000,11.539905,724.000000
50%,7857.036244,99.500000,15.655896,2149.000000,96.000000,205.000000,17.184965,1350.500000,17.009055,2042.000000,...,239.000000,121.000000,15.759071,1356.00000,2814.000000,14.142825,302.500000,156.000000,14.083167,1375.000000
75%,24090.497485,190.250000,18.022187,2886.000000,134.000000,266.000000,29.709774,2036.000000,18.591146,2785.250000,...,324.000000,173.000000,19.625128,2045.25000,2908.250000,16.016120,414.000000,221.000000,17.393545,2067.250000
max,137384.282474,2896.000000,1695.774114,5000.000000,1046.000000,1608.000000,23743.075799,5000.000000,4996.688699,5000.000000,...,1487.000000,844.000000,14306.800651,5000.00000,5000.000000,322.312371,4028.000000,2429.000000,4919.355667,5000.000000


### Error

In [41]:
df[["random_specialized_err","specialized_err","spec_tw_err", "spec_thr_err"]].describe()

,random_specialized_err,specialized_err,spec_tw_err,spec_thr_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,38.964745,21.859027,17.050575,14.744624
std,201.409312,77.936329,62.518736,10.445820
min,5.620447,5.390908,6.787472,7.074156
25%,15.155787,13.604474,13.019160,12.388319
50%,17.009055,15.655896,14.785632,14.142825
75%,18.591146,18.022187,17.061461,16.016120
max,4996.688699,1695.774114,1988.541983,322.312371


### Error Controls

In [40]:
df[["rand_match_edge_err", "match_edge_err", "tw_match_edge_err", "thr_match_edge_err"]].describe()

,rand_match_edge_err,match_edge_err,tw_match_edge_err,thr_match_edge_err
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,709.274661,276.229631,146.741324,72.924765
std,2168.554115,1336.242214,739.245195,310.641665
min,7.322908,4.479763,4.291349,5.635701
25%,16.300071,14.141500,12.788221,11.539905
50%,18.953511,17.184965,15.759071,14.083167
75%,264.511921,29.709774,19.625128,17.393545
max,25621.063399,23743.075799,14306.800651,4919.355667


### Prediction

In [48]:
df[["random_specialized_pred","specialized_pred","spec_tw_pre", "spec_thr_pre"]].describe()

,random_specialized_pred,specialized_pred,spec_tw_pre,spec_thr_pre
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,1999.188000,2420.792000,2551.899000,2648.014000
std,995.228954,868.727774,829.404268,825.589575
min,42.000000,98.000000,285.000000,281.000000
25%,1324.000000,2041.000000,2083.750000,2108.750000
50%,2042.000000,2149.000000,2744.500000,2814.000000
75%,2785.250000,2886.000000,2900.000000,2908.250000
max,5000.000000,5000.000000,5000.000000,5000.000000


### Prediction Control

In [50]:
df[["rand_match_edge_pred", "match_edge_pred", "tw_match_edge_pred", "thr_match_edge_pred"]].describe()

,rand_match_edge_pred,match_edge_pred,tw_match_edge_pred,thr_match_edge_pred
count,1000.00000,1000.000000,1000.00000,1000.000000
mean,1137.77200,1426.149000,1495.11200,1537.555000
std,925.92112,879.593553,902.49904,829.118688
min,0.00000,0.000000,0.00000,27.000000
25%,222.50000,685.000000,702.50000,724.000000
50%,1194.00000,1350.500000,1356.00000,1375.000000
75%,1953.00000,2036.000000,2045.25000,2067.250000
max,5000.00000,5000.000000,5000.00000,5000.000000


### Sizes

In [54]:
sizes = df[["rspec_sizes", "spec_sizes", "tw_sizes", "thr_sizes"]]
sizes.describe()

,rspec_sizes,spec_sizes,tw_sizes,thr_sizes
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,76.988000,117.102000,145.807000,190.788000
std,38.484658,74.618533,89.702666,144.561292
min,29.000000,33.000000,39.000000,52.000000
25%,53.000000,73.000000,89.000000,112.000000
50%,66.000000,96.000000,121.000000,156.000000
75%,89.000000,134.000000,173.000000,221.000000
max,432.000000,1046.000000,844.000000,2429.000000


### Edges

In [55]:
edges = df[["rspec_edges", "spec_edges", "tw_edges", "thr_edges"]]
edges.describe()

,rspec_edges,spec_edges,tw_edges,thr_edges
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,175.738000,236.973000,286.363000,362.180000
std,65.726565,120.622984,154.501352,248.949214
min,86.000000,86.000000,81.000000,105.000000
25%,135.000000,166.000000,192.000000,226.000000
50%,159.000000,205.000000,239.000000,302.500000
75%,196.000000,266.000000,324.000000,414.000000
max,762.000000,1608.000000,1487.000000,4028.000000


### Connectivity: (num edges)/(num nodes)^2

In [64]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
edge_prob = pd.DataFrame(np.array(edges)/(np.array(sizes)**2),columns=labels)
edge_prob.describe()

,random_specialized,specialized,spec_twice,spec_three
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.037825,0.023753,0.018371,0.013943
std,0.017372,0.012075,0.009682,0.007591
min,0.004083,0.001470,0.002087,0.000683
25%,0.024484,0.014855,0.011175,0.008418
50%,0.035973,0.022187,0.016632,0.012554
75%,0.048221,0.031007,0.023851,0.017953
max,0.107180,0.086420,0.074951,0.045267


In [66]:
df[["random_err", "random_pred"]].describe()

,random_err,random_pred
count,1000.000000,1000.000000
mean,15750.659529,183.784000
std,20722.656793,294.953391
min,14.795582,0.000000
25%,84.051292,45.000000
50%,7857.036244,99.500000
75%,24090.497485,190.250000
max,137384.282474,2896.000000
